# Les collections - Vector et HashMap

---

Une collection est un ensemble d'éléments de même type alloue dynamiquement. Les collections sont classées en 4 grandes familles :

> - **Sequences**: Vec, VecDeque, LinkedList
> - **Maps**: HashMap, BTreeMap
> - **Sets**: HashSet, BTreeSet
> - **Misc**: BinaryHeap

*Il est bien plus courant d'utiliser les Vecteurs et les HashMap que tous les autres types. Ainsi, nous n'allons ici traiter que d'eux.*

## Les Vectors

> Les vectors ou Vec sont utiles lorsque l'on a besoin de tableau redimensionable, ou que les données soient stockées sur le tas. Cependant, dans le cas d'ajout d'éléments au début du tableau, il est conseillé de passer par un autre type de collection, le vecteur est bien plus performant pour les allocations en fin de tableau que le contraire (Toutes les données nécessiteraient d'être déplacées.).

```
pub struct Vec<T, A = Global>
where
    A: Allocator,{
/* private fields */ }
```

### Creation de Vec

- Voici 3 façons de créer un nouveau Vec :
> - Avec **Vec::new()**
> - avec **to_vec()** qui est une méthode des tableaux
> - Avec la macro **vec!**

In [29]:
// A partir d'un vecteur vide
let mut v1 = Vec::new();
v1.push(1);
v1.push(2);
v1.push(3);
    
// A partir d'un tableau
let mut v2 = [1, 2, 3].to_vec();
    
// A partir de la macro vec!
let mut v3 = vec!(1, 2, 3);

*Ce ne sont pas les seuls moyens, on peut par exemple aussi utiliser **collect()** sur un iterateur*

### Utilisation de Vec

- Pour accéder aux éléments du Vecteur, l'on utilisera la **notation crochet** de la même façon que si l'on tentait d'accéder aux éléments d'un tableau. Cependant, le programme paniquera si l'on veut accéder à un index d'élément qui dépasse la taille de la collection. Il existe la méthode get() sur les vecteurs qui permet d'accéder aux éléments et qui retourne une Option, donc None si l'index était invalide :

In [8]:
{
    let v1 = vec!(1, 2, 3_u64);
    dbg!(v1[0]); // notation crochet
    dbg!(v1[1]); // notation crochet
    dbg!(v1[2]); // notation crochet

    // dbg!(v1[3]); // CETTE LIGNE PROVOQUERAIT UN PANIC
    let r = v1.get(2); // Une Option est retournee selon si l'element est present ou non
    let s = v1.get(3); // Une Option est retournee selon si l'element est present ou non
    dbg!(r);
    dbg!(s);
}

[src/lib.rs:126] v1[0] = 1
[src/lib.rs:127] v1[1] = 2
[src/lib.rs:128] v1[2] = 3
[src/lib.rs:133] r = Some(
    3,
)
[src/lib.rs:134] s = None


()

```
pub fn get<I>(&self, index: I) -> Option<&<I as SliceIndex<[T]>>::Output>
where
    I: SliceIndex<[T]>,
```
- D'après son prototype, get() ne se contente pas seulement de retourner un élément ou une référence sur ce dernier, mais il retourne une slice. Ceci peut permettre de retourner une liste d'éléments avec get() :

In [13]:
{
    let v1: Vec<u32> = (1..=10).collect(); // Creation d'un Vector contenant les nombres de 1 a 10
    let slice = v1.get(2..5);
    dbg!(slice);
}

[src/lib.rs:127] slice = Some(
    [
        3,
        4,
        5,
    ],
)


()

- Il existe différentes méthodes pour Vec, en voici quelques exemples :

In [2]:
let mut v = Vec::<u64>::new();
v.push(10); // Ajoute 10 a la fin de la colletion
dbg!(v.pop()); // Retire le dernier element de la collection et retoune une Option de celui-ci
let mut v2: Vec<u64> = vec![4, 5, 6];
v.append(&mut v2); // Copie les valeurs de v2 dans v1, vide v2
v.clear(); // Vide la collection
v.push(11);
v.iter().for_each(|v| { // Itere sur le vecteur et dump les valeurs contenues
    dbg!(v);
});
v.iter_mut().for_each(|value| { // Itere de facon mutable sur le vecteur, modifie ses valeurs
    *value *= 2;
});
dbg!(v);

[src/lib.rs:104] v.pop() = Some(
    10,
)
[src/lib.rs:110] v = 11
[src/lib.rs:115] v = [
    22,
]


### Les slices à partir d'un Vector


- Enfin, il est possible d'obtenir directement une slice de type elem à partir d'un Vec de type elem en le préfixant de l'opérateur de référencement & :

In [3]:
fn i_take_a_slice(slice: &[u8]) { // Cette fonction prend une slice en parametre
    println!("ma slice est : {:?}", slice);
    println!("sa taille est de : {}", slice.len());
}
fn now_a_u32_slice(slice: &[u32]) {
    println!("ma slice est : {:?}", slice);
    println!("sa taille est de : {}", slice.len());
}
{
    let v = vec!(1, 2, 4, 8, 16);  // Inference de Vec<u8>
    i_take_a_slice(&v);
    
    let v2 = vec!(1, 2, 4, 8, 16); // Inference de Vec<u32>
    now_a_u32_slice(&v2);
    
    let v2 = vec!(1, 2, 4, 8, 16);
    now_a_u32_slice(&v2[1..3]); // Sous slice ici !
}

ma slice est : [1, 2, 4, 8, 16]
sa taille est de : 5
ma slice est : [1, 2, 4, 8, 16]
sa taille est de : 5
ma slice est : [2, 4]
sa taille est de : 2


()

 - Naturellement, il est normal de pouvoir obtenir des sous-slices aussi :

In [17]:
{
    let v = vec!(1, 2, 4, 8, 16);
    let slice1 = &v[..2];
    i_take_a_slice(slice1);
    let slice2 = &v[0..5];
    i_take_a_slice(slice2);
    let slice3 = &v[1..v.len()];
    i_take_a_slice(slice3);
    let slice4 = &v[3..];
    i_take_a_slice(slice4);

    let v2 = vec!(1, 2, 4, 8, 16); // Inference de Vec<u32>
    let slice_u32 = &v2[3..];
    now_a_u32_slice(slice_u32);
}

ma slice est : [1, 2]
sa taille est de : 2
ma slice est : [1, 2, 4, 8, 16]
sa taille est de : 5
ma slice est : [2, 4, 8, 16]
sa taille est de : 4
ma slice est : [8, 16]
sa taille est de : 2
ma slice est : [8, 16]
sa taille est de : 2


()

- Grâce aux mécanismes de safety, un panic! se déclenche si l'on sort de la Range :

In [10]:
{
    let v = vec!(1, 2, 4, 8, 16);
    let slice = &v[..6]; // Slice trop large pour le Vector !
    i_take_a_slice(slice);
}

thread '<unnamed>' panicked at 'range end index 6 out of range for slice of length 5', src/lib.rs:28:18
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::slice::index::slice_end_index_len_fail_rt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/slice/index.rs:77:5
   3: core::slice::index::slice_end_index_len_fail
   4: <unknown>
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/slice/index.rs:69:9
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Les HashMap

- Alors que les Vecteurs seraient de la forme INDEX/VALUE, les HashMap sont plutôt de la forme KEY/VALUE. À chaque élément, est associée une clef, n'importe quel type d'élément peut être insérée tant que sa taille est connue à la compilation, mais la clef doit implémenter les traits Hash et Eq pour être utilisable :

In [20]:
use std::collections::HashMap;

#[derive(Debug)]
struct Dummy { a: usize, b: usize }
let hashmap: HashMap<usize, Dummy> = std::collections::HashMap::new(); // Creation de hashmap
dbg!(hashmap);

[src/lib.rs:112] hashmap = {}


> *HashMap ne fait pas partie du prélude Rust, ainsi, il est obligatoire de l'importer via* **use std::collections::HashMap;**

Contrairement aux vecteurs qui sont des collections linéaires, les hashmap permettent de pouvoir retrouver rapidement un élément, de le supprimer ou d'en insérer un nouveau.

- Il existe beaucoup de méthodes sur le HashMap, les plus utilisées sont insert() et get() :

In [28]:
use std::collections::HashMap;

#[derive(Debug)]
struct Dummy { a: usize, s: &'static str }
let mut hashmap: HashMap<usize, Dummy> = std::collections::HashMap::new(); // Creation de hashmap
dbg!(&hashmap);
hashmap.insert(3, Dummy{a: 254, s: "Les trois petits cochons"});
hashmap.insert(7, Dummy{a: 255, s: "Blanche neige et les septs nains"});
// dbg!(&hashmap);

dbg!(&hashmap.get(&7)); // 7 est une clef valide
dbg!(&hashmap.get(&1)); // 1 n'est pas une clef valide

[src/lib.rs:114] &hashmap = {}
[src/lib.rs:119] &hashmap.get(&7) = Some(
    Dummy {
        a: 255,
        s: "Blanche neige et les septs nains",
    },
)
[src/lib.rs:120] &hashmap.get(&1) = None


> **N'hesitez pas a tester d'autres méthodes, certaines sont vraiment sympas.**